In [57]:
import requests
import bs4
import json
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError, CollectionInvalid
import datetime as dt


def single_query(link, payload):
	response = requests.get(link, params=payload)
	if response.status_code != 200:
	    print 'WARNING', response.status_code
	else:
	    return response.json()

link = 'http://api.nytimes.com/svc/search/v2/articlesearch.json'
payload = {'api-key': 'c6461aab05fc058de08a045ba7d791d8:15:72328757','fq': 'section_name:("World" "Travel" "Science")'}

html_str = single_query(link, payload)

In [58]:
html_str

{u'copyright': u'Copyright (c) 2013 The New York Times Company.  All Rights Reserved.',
 u'response': {u'docs': [{u'_id': u'556f0c2638f0d83db8478b28',
    u'abstract': None,
    u'blog': [],
    u'byline': {u'original': u'By MANU JOSEPH',
     u'person': [{u'firstname': u'Manu',
       u'lastname': u'JOSEPH',
       u'organization': u'',
       u'rank': 1,
       u'role': u'reported'}]},
    u'document_type': u'article',
    u'headline': {u'content_kicker': u'Letter From India',
     u'kicker': u'Letter From India',
     u'main': u'In Hindi Films, Strong Women Are a Formula That Works'},
    u'keywords': [{u'is_major': u'N',
      u'name': u'subject',
      u'rank': u'1',
      u'value': u'Women and Girls'},
     {u'is_major': u'N',
      u'name': u'subject',
      u'rank': u'2',
      u'value': u'Movies'},
     {u'is_major': u'N',
      u'name': u'glocations',
      u'rank': u'3',
      u'value': u'India'}],
    u'lead_paragraph': u'The most popular Hindi films in recent months have b

In [54]:
html_str['response']['meta']['hits']

365156

In [55]:
import requests, pymongo, json, time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup


# Url for NYT dev api
NYT_URL = 'http://api.nytimes.com/svc/search/v2/articlesearch.json&fq=section_name:("World" "Travel" "Science")'

API_KEY = '81a22f3f3c6dfe5cd7d36d5a345117cd:3:66835804'

def init_mongo_client():
    # Initiate Mongo client
    client = pymongo.MongoClient()

    # Access database created for these articles
    db = client.nyt2015
    
    # Access collection created for these articles
    coll = db.articles

    # Access articles collection in db and return collection pointer.
    return db.articles


def call_api(url, payload, p=0):
    # Add the 'page' parameter to the payload.
    payload['page'] = p

    # Get the requested url. Error handling for bad requests should be done in
    # the calling function.
    return requests.get(url, params=payload)


def get_response(r):
    # Use json.loads to read the response text
    raw = json.loads(r.text)

    # Return the meta (hits, etc.) and docs (containing urls'n'stuff) back
    return raw['response']['meta'], raw['response']['docs']


def get_soup(url):
    # Header to be passed in to NYT when scraping article text.
    agent  = 'DataWrangling/1.1 (http://zipfianacademy.com; '
    agent += 'class@zipfianacademy.com)'
    headers = {'user_agent': agent}

    # Wrap in a try-except to prevent a maxTry connection error from erroring
    # out the program. Return None if there are any issues.
    try:
        r = requests.get(url, headers=headers)
    except:
        return None

    # Just in case there was a normal error returned. Pass back None.
    if r.status_code != 200: return None

    # Otherwise return a soupified object containing the url text encoded in
    # utf-8. Will toss back errors on some pages without the encoding in place.
    return BeautifulSoup(r.text.encode('utf-8'))


def get_body_text(docs):

    # Grab the url from each document, if it exists, then scrape each url for
    # its body text. If we get any errors along the way, continue on to the
    # next document / url to be scraped.
    result = []
    for d in docs:

        # Make a copy of the doc's dictionary
        doc = d.copy()

        # If there's no url (not sure why this happens sometimes) then ditch it
        if not doc['web_url']:
            continue

        # Scrape the doc's url, return a soup object with the url's text.
        soup = get_soup(doc['web_url'])
        if not soup:
            continue

        # Find all of the paragraphs with the correct class.
        # This class tag is specific to NYT articles.
        body = soup.find_all('p', class_= "story-body-text story-content")
        if not body:
            continue

        # Join the resulting body paragraphs' text (returned in a list).
        doc['body'] = '\n'.join([x.get_text() for x in body])

        print doc['web_url']
        result.append(doc)

    return result


def remove_previously_scraped(coll, docs):
    # Check to see if the mongo collection already contains the docs returned
    # from NYT. Return back a list of the ones that aren't in the collection to
    # be scraped.
    new_docs = []
    for doc in docs:
        # Check fo the document id in mongo. If it finds none, append to
        # new_docs
        cursor = articles.find({'_id': doc['_id']}).limit(1)
        if not cursor.count() > 0:
            new_docs.append(doc)

    if new_docs == []:
        return None

    return new_docs


def get_end_date(dt):
    # String-ify the datetime object to YYYMMDD, which the NYT likes.
    yr   = str(dt.year)
    mon = '0' * (2 - len(str(dt.month))) + str(dt.month)
    day = '0' * (2 - len(str(dt.day))) + str(dt.day)
    return yr + mon + day


def scrape_articles(coll, last_date):
    page = 0
    while page <= 10:
        print 'Page:', page

        # Request all of the newest articles matching the search term
        payload  = {'sort': 'newest',
                    'end_date': get_end_date(last_date),
                    'api-key': API_KEY}

        # Call the API with BaseURL + params and page
        r = call_api(NYT_URL, payload, page)

        # Increment the page before we encounter any potential errors
        page += 1

        # Check to see if the metadata didn't come back. USUALLY happens if
        # page > 100. When it does, reset the whole thing, roll the date back
        # one day, sleep for a couple seconds, then keep going.
        if r.status_code != 200:
            page = 0
            last_date += relativedelta(days=-1)
            print 'End Date:', get_end_date(last_date)
            time.sleep(2)
            continue
            
        # Get the meat data & documents from the request
        meta, docs = get_response(r)

        # Check if docs are already in the database
        new_docs = remove_previously_scraped(coll, docs)
        if not new_docs: continue

        # Grab only the docs that have these tags
        docs_with_body = get_body_text(new_docs)

        for doc in docs_with_body:
            try:
                # Insert each doc into Mongo
                coll.insert(doc)
            except:
                # If there's any error writing it in the db, just move along.
                continue

In [38]:
last_date = datetime.now() + relativedelta(days=-2)

In [39]:
last_date

datetime.datetime(2015, 6, 14, 17, 23, 46, 403286)

In [64]:
c = MongoClient()
db = c['nyt_dump']
tab = db['articles']
soup = BeautifulSoup(tab.find_one()['raw_html'], 'html.parser')


In [65]:
soup.select('.story-body-text.story-content')

[<p class="story-body-text story-content" data-para-count="231" data-total-count="231" id="story-continues-1" itemprop="articleBody">NIAMEY —  A total of 33 migrants have died in the Sahara desert in Niger while en route to Europe this year, including 18 found dehydrated last week near a road to the border with Algeria, the government of Niger said on Tuesday.</p>,
 <p class="story-body-text story-content" data-para-count="242" data-total-count="473" itemprop="articleBody">International assessments, however, have put the number closer to 50. Many thousands attempt to cross the vast and inhospitable terrain in order to reach the Libyan coast, where they hope to begin another hazardous trip by boat to Europe.</p>,
 <p class="story-body-text story-content" data-para-count="210" data-total-count="683" itemprop="articleBody">Six foreigners were found dead near a road between Agadez and the Libyan border on May 12, while nine were found dead on June 2 and four more are missing on a road to L